# Exploration of OthelloGPT

In [1]:
import os

os.environ["ACCELERATE_DISABLE_RICH"] = "1"
# os.environ["TORCH_USE_CUDA_DSA"] = "1"
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

from functools import partial
from typing import Tuple, Union

from circuitsvis.attention import attention_patterns
import einops
import torch as t
import transformer_lens.utils as utils
import wandb
from jaxtyping import Float, Int
from neel_plotly import line
from torch import Tensor
from transformer_lens import (
    HookedTransformer,
)
from transformer_lens.hook_points import HookPoint

from plotly_utils import imshow

from utils import *
from plotting import *
from probes import get_probe, get_neels_probe, ProbeTrainingArgs, LitLinearProbe

from othello_world.mechanistic_interpretability.mech_interp_othello_utils import (
    plot_single_board, )


%load_ext autoreload
%autoreload 2


pytorch_lightning not working. Cannot train probes.


## Setup

Things that you probably always want to run.

In [2]:
t.set_grad_enabled(False)
device = "cuda" if t.cuda.is_available() else "cpu"

cfg, model = get_othello_gpt(device)

Moving model to device:  cpu
Moving model to device:  cpu


In [6]:
# %% Loading sample data
full_games_tokens, full_games_board_index = load_sample_games()
num_games = 50
focus_games_tokens = full_games_tokens[:num_games]
focus_games_board_index = full_games_board_index[:num_games]
assert (TOKENS_TO_BOARD[focus_games_tokens] == focus_games_board_index).all()

focus_states = move_sequence_to_state(focus_games_board_index)
focus_valid_moves = move_sequence_to_state(focus_games_board_index, mode="valid")

print("focus states:", focus_states.shape)
print("focus_valid_moves", focus_valid_moves.shape)


Number of games: 100000
Length of game: 60
focus states: torch.Size([50, 60, 8, 8])
focus_valid_moves torch.Size([50, 60, 8, 8])


## Ablation Study

In [ ]:
individual_heads = False

n_games = 50
tokens = full_games_tokens[-n_games:].to(device)
board_index = full_games_board_index[-n_games:].to(device)
get_metrics = lambda model: get_loss(model, tokens, board_index, 5, -5).to_tensor()
zero_ablation_metrics = zero_ablation(model, get_metrics, individual_heads).cpu()
base_metrics = get_metrics(model)

In [ ]:
# Plotting the results
x_labels = [f"Head {i}" for i in range(model.cfg.n_heads)] + ["All Heads", "MLP"]
y_labels = [f"Layer {i}" for i in range(model.cfg.n_layers)]
if not individual_heads:
    x_labels = x_labels[-2:]

imshow(
    zero_ablation_metrics[:3] - base_metrics[:3, None, None],
    title="Metric increase after zeroing each component",
    x=x_labels,
    y=y_labels,
    facet_col=0,
    facet_labels=["Loss", "Cell accuracy", "Board accuracy"],
)

In [ ]:
# Abblate all attention after the layer `n`

def filter(name: str, start_layer: int = 0):
    if not name.startswith("blocks."):
        # 'hook_embed' or 'hook_pos_embed' or 'ln_final.hook_scale' or 'ln_final.hook_normalized'
        return False

    layer = int(name.split(".")[1])

    return layer >= start_layer and "attn_out" in name

metrics_per_layer = []
for start_layer in range(model.cfg.n_layers):
    with model.hooks(fwd_hooks=[(partial(filter, start_layer=start_layer),
                                    zero_ablation_hook)]):
        metrics_per_layer.append(get_metrics(model))

In [ ]:
# %% Plot
lines = t.stack(metrics_per_layer, dim=1).cpu()
line(
    lines[:3],
    x=[f"≥ {i}" for i in range(model.cfg.n_layers)],
    facet_col=0,
    #  facet_col_wrap=3,
    facet_labels=[
        "Loss",
        "Cell accuracy",
        "Board accuracy",
    ],  # 'False Positive', 'False Negative', 'True Positive', 'True Negative'],
    title="Metrics after zeroing all attention heads above a layer",
)

## Exploration of the probe

In [1]:
val_tokens, val_states = generate_training_data(n=1_000)

NameError: name 'generate_training_data' is not defined

In [4]:
# Exploration of the probe

neel_probe = get_neels_probe(False, device)
"""Shape (d_model, rows, cols, options)
options: 0: blank, 1: my piece, 2: their piece"""

blank_probe, my_probe, their_probe = neel_probe.unbind(dim=-1)
blank_direction = blank_probe - (their_probe + my_probe) / 2
my_direction = my_probe - their_probe


### Accuracy and cosine similarity

In [7]:
plot_probe_accuracy(
    model,
    neel_probe,
    val_tokens,
    per_option=True,
    # per_move="cell_accuracy",
    per_move="board_accuracy",
    name="Neel's probe",
);

In [ ]:
for probe, name in zip([blank_probe, their_probe, my_probe], ["blank", "their", "my"]):
    probe = einops.rearrange(probe, "d_model rows cols -> (rows cols) d_model")
    plot_similarities(probe,
                      title=f"Similarity between {name} vectors",
                      x=full_board_labels,
                      y=full_board_labels)

In [ ]:
# %% Similarity between mine and theirs (for each square)
plot_similarities_2(my_probe, their_probe)

In [ ]:
probes = [get_probe(i, device=device) for i in range(3)]

for probe, name in zip(probes, ["new probe", "orthogonal probe", "orthogonal probe 2"]):
    plot_probe_accuracy(
        model,
        probe.to(device),
        full_games_tokens[-100:],
        full_games_board_index[-100:],
        per_option=True,
        name=name,
    )


Similarities between the blank probe and the token embeddings

In [ ]:
token_embs = model.W_E[1:]
token_embs_64 = t.zeros((64, token_embs.shape[1]), device=token_embs.device)
token_embs_64[TOKENS_TO_BOARD] = token_embs
print(token_embs.shape)
token_embs_64 = einops.rearrange(token_embs_64, "(rows cols) d_model -> d_model rows cols", rows=8)

plot_similarities_2(
    neel_probe[..., 0],
    token_embs_64,
    name="Blank probe and token embeddings",
)

Compute and show probe vector norms

In [8]:
probe_norm = neel_probe.norm(dim=0)
px.histogram(probe_norm.cpu().flatten(), title="Probe vector norms", labels={"value": "norm"})

### Using UMAP

In [ ]:

import umap
import umap.plot
import pandas as pd

vectors = einops.rearrange(linear_probe,
                            "d_model rows cols options -> (options rows cols) d_model")

mapper = umap.UMAP(metric="cosine").fit(vectors.cpu().numpy())

labels = [probe_name for probe_name in ["blank", "their", "my"] for _ in full_board_labels]
hover_data = pd.DataFrame({
    "square": full_board_labels * 3,
    "probe": labels,
})

umap.plot.show_interactive(
    umap.plot.interactive(mapper, labels=labels, hover_data=hover_data, theme="inferno"))


### Using PCA

In [ ]:
linear_probe = get_probe(device)

# %% Run PCA on the vectors of the probe
vectors = einops.rearrange(linear_probe, "d_model rows cols options -> (options rows cols) d_model")
plot_PCA(vectors, "the probe vectors")
# %% The same be per option
for i in range(3):
    plot_PCA(
        linear_probe[..., i],
        f"the probe vectors for option {i}",
        flip_dim_order=True,
        absolute=True,
    )

# %% Normalise the probe then run PCA
normalised_probe = linear_probe / linear_probe.norm(dim=-1, keepdim=True)
plot_PCA(normalised_probe, "the normalised probe vectors", flip_dim_order=True)

# %% Same PCA but with the unembeddings
plot_PCA(model.W_U, "the unembeddings")

# %%
plot_PCA(model.W_pos, "the embeddings")
# %%
all_vectors = [
    model.W_U.T,
    model.W_E,
    model.W_pos,
    vectors,
]
all_vectors = [(v - v.mean(dim=0)) / v.std(dim=0) for v in all_vectors]

plot_PCA(t.cat(all_vectors, dim=0), "the embeddings and unembeddings")
# %%
plot_PCA(t.cat([my_direction, blank_direction], dim=1).flatten(1).T, "the direction vectors")
# %%
plot_PCA(my_direction.flatten(1).T, "the direction vectors")
# %%
plot_PCA(blank_direction.flatten(1).T, "the direction vectors")


## Training probes

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger, WandbLogger
from probes import ProbeTrainingArgs, LitLinearProbe, PROBE_DIR

In [13]:
MAKE_NEW_TRAINING_DATA = False
if MAKE_NEW_TRAINING_DATA:
    games_tokens, games_valid_moves = make_training_data()
else:
    games_tokens, games_valid_moves = get_training_data()

games_board_index = TOKENS_TO_BOARD[games_tokens]
games_states = move_sequence_to_state(games_board_index, mode="alternate")

Converting moves to states:   0%|          | 0/100 [00:00<?, ?1000 games/s]

In [ ]:
valid_tokens, _ = generate_training_data(1_000, seed=69)
valid_states = move_sequence_to_state(TOKENS_TO_BOARD[valid_tokens], mode="alternate")

### Compute stats on the dataset

In [14]:
if not STATS_PATH.exists():
    stats = compute_stats(games_states, games_valid_moves)
else:
    stats = t.load(STATS_PATH)
    print(stats.shape)

stat_names = ["Empty", "My piece", "Their piece", "Valid move"]


torch.Size([4, 60, 8, 8])


In [15]:
plot_square_as_board(
    stats.mean(1),
    title="Average frequency of each cell being ...",
    facet_col=0,
    facet_labels=stat_names,
)



In [ ]:
# %% Plot stats per cell and move
if 0:
    moves_to_show = [0, 5, 10, 20, 30, 40, 50, 55]
    x = einops.rearrange(stats[:, moves_to_show], "option m r c -> (m option) r c")
    labels = [f"{name} (move {move})" for move in moves_to_show for name in stat_names]

    plot_square_as_board(
        x,
        facet_col=0,
        facet_col_wrap=4,
        facet_labels=labels,
        title="Average frequency of each cell being ...",
        height=3000,
    )


### Actual Training

In [ ]:
probes = []

for num_probe in range(3):
    args = ProbeTrainingArgs(
        lr=1e-3,
        max_epochs=4,
        wd=0.05,
        train_tokens=games_tokens,
        train_states=games_states,
        valid_tokens=valid_tokens,
        valid_states=valid_states,
        correct_for_dataset_bias=False,
        probe_name=f"orthogonal_probe_{num_probe}",
    )
    lit_ortho_probe = LitLinearProbe(model, args, *probes)

    logger = WandbLogger(save_dir=os.getcwd() + "/logs", project=args.probe_name)
    trainer = pl.Trainer(
        max_epochs=args.max_epochs,
        logger=logger,
        log_every_n_steps=1,
        val_check_interval=100,
        check_val_every_n_epoch=None,
    )

    trainer.fit(model=lit_ortho_probe)
    probes.append(lit_ortho_probe.linear_probe)
    model = model.to(device)  # pl trainer moves model to cpu at the end :shrug:
    plot_probe_accuracy(
        model,
        lit_ortho_probe.linear_probe,
        valid_tokens,
        per_option=True,
        per_move="board_accuracy",
    )

    wandb.finish()

    path = PROBE_DIR / f"orthogonal_probe_{num_probe}.pt"
    if not path.exists():
        t.save(lit_ortho_probe.linear_probe, path)
        print(f"Saved probe to {path.resolve()}")
    else:
        print(f"Warning: {path.resolve()} already exists. Not saving the probe.")

## Constructing residual stream

In [ ]:
def make_residual_stream(
    world: Union[Int[Tensor, "row=8 cols=8"], str],
    probe: Float[Tensor, "d_model rows cols options=3"],
) -> Float[Tensor, "d_model"]:
    """
    Create the embedding of a board state according to the probe

    Args:
        world: the board state, with 0 for blank, +1 for mine, -1 for theirs
        probe: directions in the residual stream that correspond to each square.
            The last dimension is the options, with 0 for blank, 1 for mine, 2 for theirs
    """

    if isinstance(world, str):
        world = board_to_tensor(world)

    d_model = probe.shape[0]
    blank_direction = probe[..., 0] - (probe[..., 1] + probe[..., 2]) / 2
    my_direction = probe[..., 1] - probe[..., 2]

    world = world.to(probe.device)
    embedding = t.zeros(d_model, device=probe.device)
    for row in range(world.shape[0]):
        for col in range(world.shape[1]):
            if world[row, col] == 0:
                embedding += blank_direction[:, row, col]
            else:
                embedding += my_direction[:, row, col] * world[row, col]

    return embedding


# %% Try to run the model on a virtual residual stream


def hook(activation: Float[Tensor, "game move d_model"], hook: HookPoint):
    activation[:, -1] = resid


layer = 4
act_name = utils.get_act_name("resid_pre", layer)
osef_input = focus_games_tokens[:1, :20]  # 1 game, 20 moves
logits = model.run_with_hooks(osef_input, fwd_hooks=[(act_name, hook)])

# Plot what the model predicts
logits = logits_to_board(logits[0, -1], "log_prob")
plot_square_as_board(logits, title="Model predictions")


In [ ]:
board = """
........
........
........
...xo...
...ox...
........
........
........
"""

board_2 = """
........
........
........
..xxxx..
.xooooo.
.o..ox..
.x.oxo..
........
"""
board_3 = """
........
........
........
...xo...
...oo...
....o...
........
........
"""

board_tensor = board_to_tensor(board_3)
resid = make_residual_stream(board_tensor, linear_probe)
# plot_square_as_board(board_tensor)


In [ ]:
@t.inference_mode()
def modify_resid_given_probe(
        model: HookedTransformer,
        moves_orig: Int[Tensor, "move"],
        moves_new: Int[Tensor, "move"],
        *probes: Float[Tensor, "d_model rows cols options=3"],
        layer: int = 6,
        cells: Tuple[str, ...] = (),
):
    act_name = utils.get_act_name("resid_pre", layer)
    new_logits, new_cache = model.run_with_cache(
        moves_new,
        names_filter=lambda name: name == act_name,
    )

    def hook(orig_activation: Float[Tensor, "game move d_model"], hook: HookPoint):
        # Step 0. Find a basis of the subspace of the probe
        # collect the probe vectors
        all_probes = t.stack(probes, dim=-1).to(orig_activation.device)
        if cells:
            rows_cols = t.tensor([board_label_to_row_col(cell) for cell in cells])
            all_probes = all_probes[:, rows_cols[:, 0], rows_cols[:, 1]]

        probe_vectors = einops.rearrange(all_probes, "d_model ... -> (...) d_model")

        orig_activation[:, -1] = swap_subspace(
            orig_activation[:, -1],
            new_cache[act_name][:, -1],
            probe_vectors,
        )

    patched_logits = model.run_with_hooks(
        moves_orig,
        fwd_hooks=[(act_name, hook)],
    )

    # display the logits
    orig_valid_moves = move_sequence_to_state(tokens_to_board(moves_orig), mode="valid")
    if cells:
        rows_cols = [board_label_to_row_col(cell) for cell in cells]
        index = tuple(zip(*rows_cols))

        # Compute the state of orig and new board
        new_board_state = move_sequence_to_state(tokens_to_board(moves_new), mode="normal")[0, -1]
        orig_board_state = move_sequence_to_state(tokens_to_board(moves_orig), mode="normal")[0, -1]
        # Put the cells of the new board in the orig board
        orig_board_state[index] = new_board_state[index]

        valid_cells = valid_moves_from_board(orig_board_state, moves_orig.shape[1])
        new_valid_moves = one_hot(valid_cells).reshape(1, 1, 8, 8)

    else:
        new_valid_moves = move_sequence_to_state(tokens_to_board(moves_new), mode="valid")

    orig_logits = logits_to_board(model(moves_orig)[0, -1], "log_prob")
    patched_logits = logits_to_board(patched_logits[0, -1], "log_prob")
    new_logits = logits_to_board(new_logits[0, -1], "log_prob")

    scale = new_logits.abs().max().cpu()

    to_stack = [
        orig_valid_moves[0, -1] * scale,
        orig_logits,
        patched_logits,
        new_logits,
        new_valid_moves[0, -1] * scale,
        patched_logits - orig_logits,
    ]

    all_logits = t.stack([t.cpu() for t in to_stack], dim=-1)
    plot_square_as_board(
        all_logits,
        title="Model predictions",
        facet_col=-1,
        facet_col_wrap=3,
        facet_labels=[
            "New expected",
            "new logits",
            "logit diff (patch - orig)",
            "original expected",
            "original logits",
            "patched logits",
        ],
    )

    # plot_square_as_board(logits_to_board(new_logits[0, -1], 'log_prob'),
    #                      title="Model predictions (new)")
    # plot_square_as_board(logits_to_board(patched_logits[0, -1], 'log_prob'),
    #                      title="Model predictions (patched)")

    # with model.hooks(fwd_hooks=[(act_name, hook)]):
    #     plot_board_log_probs(
    #         tokens_to_board(moves_new[0]),
    #         patched_logits[0],
    #     )


orig_index = 2
new_index = 3
move_index = 20
layer = 4
orig_games = focus_games_tokens[orig_index:orig_index + 1, :move_index]
new_games = focus_games_tokens[new_index:new_index + 1, :move_index]

modify_resid_given_probe(model, orig_games, new_games, *probes, layer=layer, cells=["D2"])


In [ ]:
plot_single_board(focus_games_board_index[orig_index, :move_index], title="Original game")
plot_single_board(focus_games_board_index[new_index, :move_index], title="New game")